In [ ]:
from shapely.geometry import MultiPoint
import pandas as pd

In [ ]:
# shots and tracking df for one game
df = pd.read_csv('joint.csv', index_col=0)
shot_df = df[df['eventType'] == "SHOT"]
print(len(shot_df))

tracking_df = pd.read_json(
    "games/0042100301/0042100301_tracking.jsonl", lines=True)
print(len(tracking_df))
print(len(shot_df))

# merge the event and tracking data on the wallclock
shots_tracking_df = pd.merge(
    shot_df, tracking_df, on='wallClock', how='inner', suffixes=('_event', '_tracking'))


In [ ]:
# convex hullll

def calc_hull(row, team):
    player_coords = []
    for player in row[f'{team}Players_tracking']:
        player_coords.append(player['xyz'])
    multipoint = MultiPoint(player_coords)
    convex_hull = multipoint.convex_hull
    return convex_hull.area


shots_tracking_df['home_hull'] = shots_tracking_df.apply(
    lambda row: calc_hull(row, 'home'), axis=1)
shots_tracking_df['away_hull'] = shots_tracking_df.apply(
    lambda row: calc_hull(row, 'away'), axis=1)

print(shots_tracking_df[['home_hull', 'away_hull']].to_string())
